## 使用 assert 确保程序正确的运行

在程序运行过程中，肯会出现各种奇怪的问题。如果带着这些问题继续运行，可能导致：
1. 出现各种意想不到的的结果；
2. 结果异常，耗费大量时间去排查；
等等...

通过使用 assert 语句限制变量的范围。当变量不满足要求则直接退出程序，并提示预设的信息。

下面的代码是一段使用 assert 的示例，通过 ```assert result > 0```, 限制变量 ```result```必须要大于 0 ， 程序才能正常运行，否则就会报错并提示信息 ```Error with result = {result}```.

In [ ]:
result = 1

assert result > 0, "Error with result = {0}".format(result)

## 选择正确的逗号位置

代码版控制器对于代码的改动，通常是以行为单位。也就是说如果一行出现多处改动，会导致查看代码变动时，难以定位改动的地方。

因此在 python 中， 定义类似列表和字典的数据结构时，对其中的每个元素都是一行使用一个逗号分隔。

如下代码，展示了一个 好的逗号分隔 和 糟糕的逗号分隔：

In [ ]:
# good
hh = ['FireFly',
      'Mikasa',
      'Misaka',
      ]


# bad
hh = ['FireFly','Mikasa','Misaka',]

## python 中的下划线
- 导入模块时，开头加下划线的不会被导入
- 命名为了解决冲突可以在名字结尾加上_
- 类中使用双下划线命名或者方法，可以避免子类覆盖或访问基类的私有属性或方法

如下示例代码中定义了 ```self.__center``` ， 实例化后变成了 ```_Dock__center```, 从而防止子类和实例化后被调用。

In [2]:
class Dock:
    def __init__(self):
        self.left = 0
        self._right = 100
        self.__center = 50

    def print_center(self):
        print(self.__center)

dock = Dock()
dir(dock)

['_Dock__center',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_right',
 'left',
 'print_center']

## 格式化字符串，基于 C style

即字符串中有 '%s', '%d' 等 c 风格的格式化占位符时。在其后方添加 '%(var1, var2)' 将会用变量替换这些占位符

话不多说， 直接看例子

In [3]:
name = 'Henry'
'Hello, %s' % name

'Hello, Henry'

In [6]:
error_code = 1225
'%x' % error_code

'4c9'

In [7]:
'Hello %s, has a 0x%x error!' % (name, error_code)

'Hello Henry, has a 0x4c9 error!'

In [8]:
'Hello %(name)s, has a 0x%(error_code)x error!' % {
    'name': name,
    'error_code':error_code,
}

'Hello Henry, has a 0x4c9 error!'

## 使用模板打印字符串

使用 string 库中的 Template 可以自定义 string 模板
```python
from string import Template
template = Template('Hello, $var1!, $var2!!!')
```
$ 符号后表示的是变量， 可以自己进行定义
```python
from string import Template
class MyTemplate(Template):
    delimiter = '%'
```

In [ ]:
from string import Template

template = Template('Hello, $username!, $gg!!!')

template.substitute(username='yuukilight', gg = 123)

'Hello, yuukilight!, 123!!!'

## With 语句和上下文管理器

只要正确实现了上下文管理就可以使用 with 语句。
### ```__enter__``` 与 ```__exit__```

上下文管理器通过 ```__enter__``` 和 ```__exit__``` 两个方法实现

例如：
```python
class Query(object):
    def __init__(self, name):
        self.name = name
    
    def __enter__(self):
        print('Begin')
        return self

    def __exit__(self, exc_type, exc_value, trace_back):
        if exc_type:
            print('Error')
        else:
            print('End')

    def query(self):
        print('Query info about %s...' % self.name)

with Query('Bob') as q:
    q.query()
```

In [3]:
class Query(object):
    def __init__(self, name):
        self.name = name
    
    def __enter__(self):
        print('Begin')
        return self

    def __exit__(self, exc_type, exc_value, trace_back):
        if exc_type:
            print('Error')
        else:
            print('End')

    def query(self):
        print('Query info about %s...' % self.name)

with Query('Bob') as q:
    q.query()

Begin
Query info about Bob...
End


编写__enter__和__exit__仍然很繁琐，因此Python的标准库contextlib提供了更简单的写法，上面的代码可以改写如下：
```python
from contextlib import contextmanager

class Query(object):
    def __init__(self, name):
        self.name = name

    def query(self):
        print('Query info about %s...' % self.name)

@contextmanager
def create_query(name):
    print('Begin')
    q = Query(name)
    yield q
    print('End')
```

@contextmanager 这个 decorator 接受一个generator，用yield语句把with ... as var把变量输出出去，然后，with语句就可以正常地工作了：

相当于 yield 返回前的部分是 ```__enter__```, yield 返回后的部分是 ```__exit__```. 

### @contextmanager 

当我们希望在执行某段代码前后固定执行特定代码，也可以使用 @contextmanager 实现。
例如：

In [4]:
from contextlib import contextmanager

@contextmanager
def tag(name):
    print("<%s>" % name)
    yield
    print("<%s>" % name)

with tag('yuukilight'):
    print('Hello')
    print('world')

<yuukilight>
Hello
world
<yuukilight>


代码执行顺序
1. with 语句首先执行 yield 前的语句
2. 执行 with 内部所有的语句
3. 最后执行 yield 后的语句

### @closing

如果一个对象没有实现上下文，我们就不能把它用于with语句。这个时候，可以用closing()来把该对象变为上下文对象。例如，用with语句使用urlopen()：
```python
from contextlib import closing
from urllib.request import urlopen

with closing(urlopen('https://www.python.org')) as page:
    for line in page:
        print(line)

```
closing也是一个经过@contextmanager装饰的generator，这个generator编写起来其实非常简单：
```python
@contextmanager
def closing(thing):
    try:
        yield thing
    finally:
        thing.close()
```
@closing 的作用是将任意对象变成上下文对象，从而可以支持 with 语句。


## Comment
### block comments
- 通常多行，解释一些细节。
- 缩进和当前代码同级。
- 注释如果要分段，使用空白行间隔。空白行同样需要 # 号开头。
- 块注释中不要去解释过程

### inline comments
- 尽量避免使用
- 尽量和描述的代码在同一行
- 和代码之间最好有两个以上的空格
- 避免用来描述很明显的逻辑

注释要和代码保持同步更新


In [10]:
# Add comment for you codes
# Block comment tutrial

from typing import Iterable

def read_lines_from_file(path: str) -> Iterable[str]:
    # Read every line from the given file,
    # if just handles the rows which is not blank,
    # it should ignore the newline character
    #
    # By the way, it should not check the existence of the given file
    # it's not the business of this function
    # let the invoker to handle the raises if it does not exists that file
    #
    # the file readline document below
    # see: https://jupyter-docker-stacks..................................................................................................................................................
    with open(path, 'r') as file:
        while line := file.readline():
            line = line.strip()
            if line:
                yield line

In [11]:
import this

The Zen of Python, by Tim Peters

Beautiful is better than ugly.
Explicit is better than implicit.
Simple is better than complex.
Complex is better than complicated.
Flat is better than nested.
Sparse is better than dense.
Readability counts.
Special cases aren't special enough to break the rules.
Although practicality beats purity.
Errors should never pass silently.
Unless explicitly silenced.
In the face of ambiguity, refuse the temptation to guess.
There should be one-- and preferably only one --obvious way to do it.
Although that way may not be obvious at first unless you're Dutch.
Now is better than never.
Although never is often better than *right* now.
If the implementation is hard to explain, it's a bad idea.
If the implementation is easy to explain, it may be a good idea.
Namespaces are one honking great idea -- let's do more of those!


## 布局
- 不同类和函数之间用两个空行隔开
- 类内部的函数中一个空行隔开
- 函数内可以划分为不同步骤，不同的步骤可以分开

## 文档字符串（Docstring）
我们可以在函数体的第一行使用一对三个单引号 ''' 或者一对三个双引号 """ 来定义文档字符串。

你可以使用 __doc__（注意双下划线）调用函数中的文档字符串属性。

DocStrings 文档字符串使用惯例：它的首行简述函数功能，第二行空行，第三行为函数的具体描述。

注意不要用这种方式写注释，因为会导致一些自动化工具自动解析成文档。导致阅读困难

**Ex 1:**

In [12]:
def execute():
    """执行操作
    
    先执行a, 再执行b
    """
    pass

print(execute.__doc__)

执行操作
    
    先执行a, 再执行b
    


In [23]:
execute?

Signature: execute()
Docstring:
执行操作

先执行a, 再执行b
File:      c:\users\b3720\appdata\local\temp\ipykernel_15976\3133771763.py
Type:      function

**Ex 2:**

In [9]:
def fetch_content_from(url: str) -> str:
    """fetch_content_from fetch the content in utf-8 encoded with given url
    Some decription here.
    Another decription here.

    Not: The key point to note

    Args:
        url: A string where to fetch the content from

    Returns:
        A string with utf-8 encoded content provided by the site

    Raises:
        ValueError: if url argument is not of the expected type(str)
    """
    pass

In [17]:

fetch_content_from?

Signature: fetch_content_from(url: str) -> str
Docstring:
fetch_content_from fetch the content in utf-8 encoded with given url
Some decription here.
Another decription here.

Not: The key point to note

Args:
    url: A string where to fetch the content from

Returns:
    A string with utf-8 encoded content provided by the site

Raises:
    ValueError: if url argument is not of the expected type(str)
File:      c:\users\b3720\appdata\local\temp\ipykernel_15976\3322123609.py
Type:      function

In [11]:
fetch_content_from.__doc__

'fetch_content_from fetch the content in utf-8 encoded with given url\n    Some decription here.\n    Another decription here.\n\n    Not: The key point to note\n\n    Args:\n        url: A string where to fetch the content from\n\n    Returns:\n        A string with utf-8 encoded content provided by the site\n\n    Raises:\n        ValueError: if url argument is not of the expected type(str)\n    '

**Ex 3:**

In [24]:
dict.update?

Docstring:
D.update([E, ]**F) -> None.  Update D from dict/iterable E and F.
If E is present and has a .keys() method, then does:  for k in E: D[k] = E[k]
If E is present and lacks a .keys() method, then does:  for k, v in E: D[k] = v
In either case, this is followed by: for k in F:  D[k] = F[k]
Type:      method_descriptor

PEP 3107
## 注解 Annotations



In [25]:
class Circle:
    def __init__(self, radius):
        self.radius = radius
        self.area = 0

def area(radius: float) -> Circle:
    """Comput area of a Cricle with given radius"""
    pass

area.__annotations__

{'radius': float, 'return': __main__.Circle}

## 命名
一般情况下

描述业务功能 优于 函数功能。好的命名不需要注释